# Task: Real-Time Pose Detection with Movenet

In [1]:
%pip install matplotlib
%pip install tensorflow==2.4.1 
%pip install opencv-python

     ---------------------------------------- 8.0/8.0 MB 1.4 MB/s eta 0:00:00
     -------------------------------------- 188.2/188.2 kB 2.8 MB/s eta 0:00:00
  Using cached cycler-0.12.1-py3-none-any.whl (8.3 kB)
     ---------------------------------------- 2.2/2.2 MB 1.6 MB/s eta 0:00:00
  Using cached kiwisolver-1.4.5-cp311-cp311-win_amd64.whl (56 kB)
  Using cached numpy-1.26.4-cp311-cp311-win_amd64.whl (15.8 MB)
     ---------------------------------------- 2.5/2.5 MB 1.6 MB/s eta 0:00:00
     -------------------------------------- 103.2/103.2 kB 2.0 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement tensorflow==2.4.1 (from versions: 2.12.0rc0, 2.12.0rc1, 2.12.0, 2.12.1, 2.13.0rc0, 2.13.0rc1, 2.13.0rc2, 2.13.0, 2.13.1, 2.14.0rc0, 2.14.0rc1, 2.14.0, 2.14.1, 2.15.0rc0, 2.15.0rc1, 2.15.0, 2.15.1, 2.16.0rc0, 2.16.1)
ERROR: No matching distribution found for tensorflow==2.4.1

[notice] A new release of pip available: 22.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


  Using cached opencv_python-4.9.0.80-cp37-abi3-win_amd64.whl (38.6 MB)
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import tensorflow as tf
import numpy as np
from matplotlib import pyplot as plt
import cv2

___

In [4]:
interpreter = tf.lite.Interpreter(model_path='4.tflite')
interpreter.allocate_tensors()

In [5]:
interpreter.get_input_details()

[{'name': 'serving_default_input:0',
  'index': 0,
  'shape': array([  1, 192, 192,   3]),
  'shape_signature': array([  1, 192, 192,   3]),
  'dtype': numpy.uint8,
  'quantization': (0.0, 0),
  'quantization_parameters': {'scales': array([], dtype=float32),
   'zero_points': array([], dtype=int32),
   'quantized_dimension': 0},
  'sparsity_parameters': {}}]

In [6]:
interpreter.get_output_details()

[{'name': 'StatefulPartitionedCall:0',
  'index': 316,
  'shape': array([ 1,  1, 17,  3]),
  'shape_signature': array([ 1,  1, 17,  3]),
  'dtype': numpy.float32,
  'quantization': (0.0, 0),
  'quantization_parameters': {'scales': array([], dtype=float32),
   'zero_points': array([], dtype=int32),
   'quantized_dimension': 0},
  'sparsity_parameters': {}}]

___

In [7]:
def draw_keypoints(frame, keypoints, confidence_threshold):
    y, x, c = frame.shape
    shaped = np.squeeze(np.multiply(keypoints, [y,x,1])) # squeeze will remove the extra dimensions.
    # need to convert the y and x coordinates to int, but by casting, we should not convert the confidence to int. So we will do that later.

    
    for kp in shaped:
        ky, kx, kp_conf = kp
        if kp_conf > confidence_threshold:
            cv2.circle(frame, (int(kx), int(ky)), 4, (0,255,0), -1)  # how big:4, color of the circles:(0,255,0), how thich our circle should be:-1 =fills the circle with that color.
            


In [8]:
def draw_connections(frame, keypoints, edges, confidence_threshold):
    y, x, c = frame.shape
    shaped = np.squeeze(np.multiply(keypoints, [y,x,1]))
    
    for edge, color in edges.items():
        p1, p2 = edge
        y1, x1, c1 = shaped[p1]
        y2, x2, c2 = shaped[p2]
        
        if (c1 > confidence_threshold) & (c2 > confidence_threshold):      
            cv2.line(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0,0,255), 2)


In [9]:
EDGES = {
    (0, 1): 'm',
    (0, 2): 'c',
    (1, 3): 'm',
    (2, 4): 'c',
    (0, 5): 'm',
    (0, 6): 'c',
    (5, 7): 'm',
    (7, 9): 'm',
    (6, 8): 'c',
    (8, 10): 'c',
    (5, 6): 'y',
    (5, 11): 'm',
    (6, 12): 'c',
    (11, 12): 'y',
    (11, 13): 'm',
    (13, 15): 'm',
    (12, 14): 'c',
    (14, 16): 'c'
}

* Those 17 landmarks are respectively in order as follows:
* nose, left eye, right eye, left ear, right ear, left shoulder, right shoulder, left elbow, right elbow, left wrist, right wrist, left hip, right hip, left knee, right knee, left ankle, right ankle

In [10]:
cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, frame = cap.read()
    img = frame.copy()
    img = tf.image.resize_with_pad(np.expand_dims(img,axis=0),192,192)
    input_image = tf.cast(img,dtype=tf.uint8)
    
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()
    
    interpreter.set_tensor(input_details[0]['index'],np.array(input_image))
    interpreter.invoke()
    keypoints_with_scores = interpreter.get_tensor(output_details[0]['index'])
    print(keypoints_with_scores)
    draw_connections(frame,keypoints_with_scores,EDGES,0.4)
    draw_keypoints(frame,keypoints_with_scores,0.4)

    # Check if the frame is valid
    if not ret:
        print("Error: Unable to capture frame")
        break
    
    # Check if the size of the frame is valid
    if frame.shape[0] <= 0 or frame.shape[1] <= 0:
        print("Error: Invalid frame size")
        break
    
    cv2.imshow ('Movenet Lightening', frame)
    
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


[[[[0.7017864  0.5343228  0.4654684 ]
   [0.6677663  0.5636316  0.5333419 ]
   [0.6728952  0.49473354 0.65410745]
   [0.6794027  0.60345024 0.4197625 ]
   [0.6984523  0.42397553 0.50520945]
   [0.7595938  0.63857687 0.3827543 ]
   [0.81329864 0.3854998  0.29449746]
   [0.86427486 0.68913275 0.13055322]
   [0.8748396  0.31584767 0.09193221]
   [0.861913   0.6293718  0.23304564]
   [0.83282375 0.43048015 0.14233229]
   [0.52328306 0.56588626 0.14557898]
   [0.64809084 0.39709342 0.06179284]
   [0.33658    0.61757654 0.3078635 ]
   [0.7067289  0.40078235 0.11572243]
   [0.34645832 0.6319189  0.18861337]
   [0.6993064  0.39954513 0.07432435]]]]
[[[[0.6988204  0.53163147 0.49044317]
   [0.66443473 0.5628423  0.48651066]
   [0.6702742  0.49319923 0.5390004 ]
   [0.68430823 0.60335195 0.4031922 ]
   [0.7006778  0.4239758  0.5204015 ]
   [0.7811253  0.64448845 0.26867762]
   [0.8384064  0.36922407 0.35077742]
   [0.8755579  0.7101481  0.1055976 ]
   [0.8861153  0.31240734 0.09318209]
   [0.848

___